In [14]:
import pandas as pd
import numpy as np
import pyodbc;
from sklearn.model_selection import GridSearchCV, train_test_split, model_selection
from sklearn.cross_validation import StratifiedKFold

from IPython.core.display import display,HTML
display(HTML("<style>.container{width:90% !important;}</style>"))

conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};'r'SERVER=WIN-5G5AUOCEJPK;'r'DATABASE=Gold585;'r'Trusted_Connection=yes;')

ImportError: cannot import name 'model_selection'

In [2]:
sql_query = "Select PartnerID \
            ,QntPurchaseCode,labels,Kolco,SERGI,PPODVES,FUTLAR,CEP,BRASLET,KOLE,UNKNOWN_WEAR \
            ,NextItem \
            from RetailCommonNext \
            where QntPurchaseCode = 3  and NextItem Is Not Null \
            order By PartnerID, PurchaseDateTime"
data = pd.read_sql(sql_query,conn)

In [3]:
data.head()

,PartnerID,QntPurchaseCode,labels,Kolco,SERGI,PPODVES,FUTLAR,CEP,BRASLET,KOLE,UNKNOWN_WEAR,NextItem
0,4,3,2.3,2,1,0,0,0,0,0,0,CEP
1,12,3,2.2,1,0,1,0,0,1,0,0,SERGI
2,20,3,2.2,0,2,0,0,2,0,0,0,Kolco
3,39,3,2.2,2,1,0,0,0,0,0,0,BRASLET
4,44,3,2.2,3,0,1,2,0,0,0,0,BRASLET


In [4]:
data.shape

(330499, 12)

In [5]:
label_map = {'Kolco':0, 'SERGI':1, 'PPODVES':2, 'CEP':3, 'BRASLET':4, 'KOLE': 5,'UNKNOWN_WEAR': 6}
data['NextItem'] = data['NextItem'].map(label_map)

In [6]:
data.head()

,PartnerID,QntPurchaseCode,labels,Kolco,SERGI,PPODVES,FUTLAR,CEP,BRASLET,KOLE,UNKNOWN_WEAR,NextItem
0,4,3,2.3,2,1,0,0,0,0,0,0,3
1,12,3,2.2,1,0,1,0,0,1,0,0,1
2,20,3,2.2,0,2,0,0,2,0,0,0,0
3,39,3,2.2,2,1,0,0,0,0,0,0,4
4,44,3,2.2,3,0,1,2,0,0,0,0,4


In [7]:
data.labels.value_counts()

2.2    88804
2.3    76705
2.4    70777
2.5    68019
2.1    11679
3.1     6561
3.2     5333
3.3     2621
Name: labels, dtype: int64

In [8]:
data = pd.get_dummies(data,columns = ['labels'], drop_first=True)#, dummy_na=True)

In [9]:
data.head()

,PartnerID,QntPurchaseCode,Kolco,SERGI,PPODVES,FUTLAR,CEP,BRASLET,KOLE,UNKNOWN_WEAR,NextItem,labels_2.2,labels_2.3,labels_2.4,labels_2.5,labels_3.1,labels_3.2,labels_3.3
0,4,3,2,1,0,0,0,0,0,0,3,0,1,0,0,0,0,0
1,12,3,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0
2,20,3,0,2,0,0,2,0,0,0,0,1,0,0,0,0,0,0
3,39,3,2,1,0,0,0,0,0,0,4,1,0,0,0,0,0,0
4,44,3,3,0,1,2,0,0,0,0,4,1,0,0,0,0,0,0


In [10]:
X = data.drop('NextItem',axis=1)
y = pd.DataFrame(data['NextItem'])

In [11]:
y.NextItem.value_counts()

0    108046
1     82842
2     59798
3     44433
4     30666
5      4596
6       118
Name: NextItem, dtype: int64

In [12]:
import xgboost as xgb
from sklearn.metrics import fbeta_score, make_scorer,accuracy_score
from sklearn import metrics

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 123,test_size=0.25,stratify =y)

In [16]:
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

In [81]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 10
param['silent'] = 1
param['nthread'] = 5
param['num_class'] = 7

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)

[0]	train-merror:0.63047	test-merror:0.644369
[1]	train-merror:0.630244	test-merror:0.643825
[2]	train-merror:0.630352	test-merror:0.643655
[3]	train-merror:0.630114	test-merror:0.643474
[4]	train-merror:0.629941	test-merror:0.643231


In [91]:
error_rate = np.sum(pred != y_test.NextItem) / y_test.NextItem.shape[0]
print((error_rate))

0.643231467473525


In [93]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bst.predict(xg_test).reshape(y_test.NextItem.shape[0], 7)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != y_test.NextItem) / y_test.NextItem.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-merror:0.63047	test-merror:0.644369
[1]	train-merror:0.630244	test-merror:0.643825
[2]	train-merror:0.630352	test-merror:0.643655
[3]	train-merror:0.630114	test-merror:0.643474
[4]	train-merror:0.629941	test-merror:0.643231
Test error using softprob = 0.643231467473525


In [110]:
xgb_model = xgb.XGBClassifier()
#scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}
parameters = {'nthread':[1,3,5], #when use hyperthread, xgboost may become slower
              'objective':['multi:softprob'],
              'learning_rate':[0.05,0.1], #so called `eta` value
              'max_depth': [5,10,15],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.8],
              'n_estimators': [25,30,35], #number of trees, change it to 1000 for better results
              'missing':[-1],
              'seed': [1337]}


clf = GridSearchCV(xgb_model, parameters, n_jobs=30, 
                   cv=StratifiedKFold(y_train.NextItem, n_folds=5, shuffle=True), 
                   #scoring=scoring,
                   verbose=2, refit='multi:softprob')
clf.fit(X_train, y_train)


best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

test_probs = clf.predict_proba(X_test)[:,1]
print(metrics.classification_report(y_test, test_probs))
print(metrics.confusion_matrix(y_test, test_probs))
print(roc_auc_score(y_test, test_probs))

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=30)]: Done 102 tasks      | elapsed: 12.5min
[Parallel(n_jobs=30)]: Done 270 out of 270 | elapsed: 35.1min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Raw AUC score: 0.35910583602959567
colsample_bytree: 0.8
learning_rate: 0.1
max_depth: 5
min_child_weight: 11
missing: -1
n_estimators: 35
nthread: 1
objective: 'multi:softprob'
seed: 1337
silent: 1
subsample: 0.8


NameError: name 'metrics' is not defined

In [122]:
test_probs = bst.predict(X_test)

AttributeError: 'DataFrame' object has no attribute 'feature_names'

In [123]:
print(metrics.classification_report(y_test.NextItem, pred_label))

             precision    recall  f1-score   support

          0       0.37      0.74      0.49     27012
          1       0.35      0.32      0.34     20711
          2       0.31      0.14      0.19     14950
          3       0.28      0.05      0.08     11108
          4       0.07      0.00      0.00      7666
          5       0.21      0.01      0.02      1149
          6       0.00      0.00      0.00        29

avg / total       0.31      0.36      0.29     82625



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)
error_rate_last = 1
parametrs = {}
# use softmax multi-class classification
parametrs['objective'] = 'multi:softmax'
parametrs['silent'] = 1
parametrs['num_class'] = 7
# scale weight of positive examples
i=1
for eta in [0.01,0.005,0.05]:
    for max_depth in [5,10,15]:
            for nthread in [5,15,30]:
                for n_estimators in [20,50,100]:
                    parametrs['eta'] = eta;parametrs['max_depth'] = max_depth;parametrs['nthread'] = nthread;
                    bst = xgb.train(parametrs, xg_train, 50, watchlist) 
                    pred = bst.predict(xg_test)
                    error_rate = np.sum(pred != y_test.NextItem) / y_test.NextItem.shape[0]
                    if error_rate < error_rate_last:
                        best_model = bst
                        error_rate_last = error_rate
                        print(i,error_rate)

                    if i%10 ==0:
                        print(i,best_parameters)
                    i = i+1


In [126]:
param

{'objective': 'multi:softmax',
 'silent': 1,
 'num_class': 7,
 'eta': 0.05,
 'max_depth': 15,
 'nthread': 30}